In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 그래프를 그리기 위한 라이브러리 설치
# !pip install -qU grandalf

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
vectorstore = Chroma.from_texts(
    [
        "재호는 랭체인 주식회사에서 근무를 하였습니다.",
        "재호의 직업은 RAG개발자입니다.",
        "진원의 직업은 음악치료사입니다.",
        "재호는 로또 1등에 당첨되었습니다.",
        "재호는 진원이와 곧 결혼합니다.",
    ],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

In [4]:
retriever = vectorstore.as_retriever()
template = """
Answer the questio based only on the following context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(
    template
)

model = ChatOpenAI(model="gpt-4o-mini")

In [5]:
chain = (
    {"context":retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
chain.invoke("재호는 누구랑 결혼해?")

'재호는 진원이와 결혼합니다.'

# 그래프 구성 확인
`Runnable` 그래프를 얻을 수 있다.

In [8]:
# 체인의 그래프에서 노드를 가져온다.
chain.get_graph().nodes

{'788da180c77c44b49bd27fdf083407d5': Node(id='788da180c77c44b49bd27fdf083407d5', name='Parallel<context,question>Input', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Input'>, metadata=None),
 'a2339931e22c4df7b9cd73ea1603227b': Node(id='a2339931e22c4df7b9cd73ea1603227b', name='Parallel<context,question>Output', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Output'>, metadata=None),
 'e01a63000f704b7d93298dacb66b3851': Node(id='e01a63000f704b7d93298dacb66b3851', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001490FEBE950>), metadata=None),
 '465f0f3384924700a9a29e407128ee19': Node(id='465f0f3384924700a9a29e407128ee19', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '7e8573c4614f48f786e2ad8a4cbc586d': Node(id='7e8573c4614f48f786e2ad8a4cbc586d', name='ChatPromptTemplate', data=ChatPromptTemplate(input_variables=['co

In [9]:
# 체인의 그래프에서 엣지를 가져온다
chain.get_graph().edges

[Edge(source='2d24583c45044d028606fa73b74d8214', target='95ef06a0eaf34380aaac97b8137d070c', data=None, conditional=False),
 Edge(source='95ef06a0eaf34380aaac97b8137d070c', target='b8782630b9be49ca964c278f6794ac07', data=None, conditional=False),
 Edge(source='2d24583c45044d028606fa73b74d8214', target='acf0a90b535e4143a12c46a1a2b08ddc', data=None, conditional=False),
 Edge(source='acf0a90b535e4143a12c46a1a2b08ddc', target='b8782630b9be49ca964c278f6794ac07', data=None, conditional=False),
 Edge(source='b8782630b9be49ca964c278f6794ac07', target='89f9eff9efc04dcf80fa2da78abb53f8', data=None, conditional=False),
 Edge(source='89f9eff9efc04dcf80fa2da78abb53f8', target='aff983690f1b4d82bba86c376c009311', data=None, conditional=False),
 Edge(source='5fc2c7f050a84ecc8ee71b3b65432e06', target='3764998335404ca4bb12ab4ef510cf2f', data=None, conditional=False),
 Edge(source='aff983690f1b4d82bba86c376c009311', target='5fc2c7f050a84ecc8ee71b3b65432e06', data=None, conditional=False)]

# 그래프출력
그래프를 출력하면 이해하기 쉬운 형태로 표현할 수 있다.

`chain.get_graph().print_ascii()`

In [10]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

# 프롬프트 가져오기
`chain.get_prompts()`

In [11]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the questio based only on the following context:\n{context}\n\nQuestion:\n{question}\n'))])]

True